In [5]:
import pandas as pd

In [55]:
article=pd.read_table('data/gender_ratio/AntiDefenseFinder_biblio.csv',sep=';',encoding = "ISO-8859-1")
article['Author_list']=article.Author.map(lambda x: x.split(";"))
Author=pd.DataFrame({'First':[],'Second':[],"Rank":[],"Title":[]})
for i in article.index.to_list():
    liste=article.Author_list[i]
    n=0
    article_name=article.Title[i]
    for j in range(len(liste)):
        last=article.Author_list[i][j].split(",")[0]
        first=article.Author_list[i][j].split(", ")[1].split(" ")[0]
        if n==0:
            rank='first'
        elif n==len(liste)-1:
            rank='last'
        else:
            rank='mid'
        author=pd.DataFrame({'First':[first],'Second':[last],'Rank':[rank],'Title':[article_name]})
        Author=pd.concat([Author,author])
        n+=1
        
        
len(article)

70

In [56]:
from unidecode import unidecode

Author['First']=Author.First.map(unidecode)
Author['First']=Author.First.map(lambda x: x.lower())
Author['Second']=Author.Second.map(unidecode)
Author['Second']=Author.Second.map(lambda x: x.lower())
Author['First']=Author.First.map(lambda x: x.replace(" ",""))
Author['First']=Author.First.map(lambda x: x.replace("-",""))
Author=Author.reset_index(drop=True)

In [57]:
name_gender=pd.read_table('data/gender_ratio/name_gender_dataset.csv',sep=',')
name_gender['Name']=name_gender.Name.map(unidecode)
name_gender['Name']=name_gender.Name.map(lambda x: x.lower())
name_gender['Name']=name_gender.Name.map(lambda x: x.replace(" ",""))
name_gender['Name']=name_gender.Name.map(lambda x: x.replace("-",""))
name_gender=name_gender.pivot_table(index='Name',columns='Gender',values='Count',aggfunc=sum).fillna(0)
name_gender['Name']=name_gender.index
name_gender['Name']=name_gender.index
name_gender['Ratio_M']=name_gender.M/(name_gender.M+name_gender.F)
name_gender['Total_name']=name_gender.M+name_gender.F
name_gender=name_gender.loc[name_gender.Total_name>5]
name_gender['Gender']="U"
name_gender.loc[name_gender.Ratio_M>0.9,'Gender']='M'
name_gender.loc[name_gender.Ratio_M<0.10,'Gender']='F'
name_gender.reset_index(drop=True,inplace=True)

/tmp/ipykernel_100/1030746288.py:6: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  name_gender=name_gender.pivot_table(index='Name',columns='Gender',values='Count',aggfunc=sum).fillna(0)
/tmp/ipykernel_100/1030746288.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_gender['Gender']="U"


In [58]:
for i in Author.index.to_list():
    Author.loc[i,'isin_db']=len(name_gender.loc[name_gender.Name==Author.First[i]])
Author

,First,Second,Rank,Title,isin_db
0,f.,tesson,first,A Comprehensive Resource for Exploring Antipha...,0.0
1,r.,planel,mid,A Comprehensive Resource for Exploring Antipha...,0.0
2,a.,egorov,mid,A Comprehensive Resource for Exploring Antipha...,0.0
3,h.,georjon,mid,A Comprehensive Resource for Exploring Antipha...,0.0
4,h.,vaysset,mid,A Comprehensive Resource for Exploring Antipha...,0.0
...,...,...,...,...,...
585,roger,buxton,last,Cyclic-AMP and bacterial cyclic-AMP receptor p...,1.0
586,ja1/4rgen,linder,first,cGMP production in bacteria,0.0
587,jason,nomburg,first,Birth of new protein folds and functions in th...,1.0
588,nathan,price,mid,Birth of new protein folds and functions in th...,1.0


In [59]:
Author0=Author.loc[Author.isin_db==0,]
Author=Author.loc[Author.isin_db!=0,]
Author=Author.merge(name_gender[['Name','Gender']],left_on='First',right_on='Name')


In [60]:
Author0

,First,Second,Rank,Title,isin_db
0,f.,tesson,first,A Comprehensive Resource for Exploring Antipha...,0.0
1,r.,planel,mid,A Comprehensive Resource for Exploring Antipha...,0.0
2,a.,egorov,mid,A Comprehensive Resource for Exploring Antipha...,0.0
3,h.,georjon,mid,A Comprehensive Resource for Exploring Antipha...,0.0
4,h.,vaysset,mid,A Comprehensive Resource for Exploring Antipha...,0.0
...,...,...,...,...,...
555,rishub,jain,mid,Highly accurate protein structure prediction w...,0.0
567,oriol,vinyals,mid,Highly accurate protein structure prediction w...,0.0
570,pushmeet,kohli,mid,Highly accurate protein structure prediction w...,0.0
574,edze,westra,mid,Prophages mediate defense against phage infect...,0.0


In [61]:
len(Author0.loc[Author0.Rank.isin(['last','first'])])

32

In [62]:
Author0=Author0.loc[Author0.Rank.isin(['last','first'])].reset_index(drop=True)
Author0

,First,Second,Rank,Title,isin_db
0,f.,tesson,first,A Comprehensive Resource for Exploring Antipha...,0.0
1,j.,cury,last,A Comprehensive Resource for Exploring Antipha...,0.0
2,linyi,gao,first,Prokaryotic innate immunity through pattern re...,0.0
3,s.,bari,first,A unique mode of nucleic acid immunity perform...,0.0
4,nitzan,tal,first,Bacteria deplete deoxynucleotides to defend ag...,0.0
5,pu,gao,last,Molecular basis of RADAR anti-phage supramolec...,0.0
6,zhangfei,shen,first,Assembly-mediated activation of the SIR2-HerA ...,0.0
7,tianmin,fu,last,Assembly-mediated activation of the SIR2-HerA ...,0.0
8,dongmei,tang,first,Multiple enzymatic activities of a Sir2-HerA s...,0.0
9,yamei,yu,last,Multiple enzymatic activities of a Sir2-HerA s...,0.0


In [63]:
Author0=Author0.loc[Author0.Rank.isin(['last','first'])].reset_index(drop=True)
Author0['Gender']=['M', 'M', 'M', 'M', 'F',
                  'M', 'M', 'M' ,'F' , 'F',
                  'M','M', 'F', 'M', 'F',
                  'F', 'F', 'M', 'F', 'F',
                  'M', 'M', 'U', 'M', 'M',
                  'M', 'M', 'M', 'M', 'M',
                  'M','M']

In [64]:
author_tot=pd.concat([Author,Author0])
author_tot=author_tot.loc[author_tot.Rank.isin(['first','last'])]
author_tot.loc[(author_tot.First=='rotem')&(author_tot.Second==' sorek'),'Gender']='M'
author_tot.loc[(author_tot.First=='adi')&(author_tot.Second=='millman'),'Gender']='F'
author_tot.loc[(author_tot.First=='gal')&(author_tot.Second=='ofir'),'Gender']='M'
author_tot.loc[(author_tot.First=='yue')&(author_tot.Second==' feng'),'Gender']='F'
author_tot.loc[(author_tot.First=='ilya')&(author_tot.Second=='osterman'),'Gender']='M'
author_tot.loc[(author_tot.First=='yuchen')&(author_tot.Second=='yan'),'Gender']='M'
author_tot.loc[(author_tot.First=='emmanuelle')&(author_tot.Second==' charpentier'),'Gender']='F'
author_tot.loc[(author_tot.First=='enea')&(author_tot.Second=='maffei'),'Gender']='M'


In [65]:
author_tot.groupby('Rank').Gender.value_counts(normalize=True)

Rank   Gender
first  M         0.685714
       F         0.314286
last   M         0.820896
       F         0.164179
       U         0.014925
Name: proportion, dtype: float64

In [ ]:
author_tot